In [1]:
import pandas as pd
df = pd.read_csv('deepsea.csv')
df.head()


,tweet,category
0,Fascinating discovery! An abyssal fish species...,biology
1,Deep sea fish really said 'surface world I'm c...,meme
2,Research suggests rapid vertical migration mig...,biology
3,lmao this fish literally speedran evolving to ...,meme
4,The vertical migration observed in this abyssa...,biology


In [ ]:
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()